In [1]:
!pip install gcloud

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.4/454.4 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gcloud: filename=gcloud-0.18.3-py3-none-any.whl size=602926 sha256=6e528a9f570d42a717c2132dd78f44f0a91f8be22b2a26cb9655db2839f78b7f
  Stored in directory: /root/.cache/pip/wheels/7c/30/88/5017af921da3a33af785f0d0fd3e944b845bc62a445a2c2f69
Successfully built gcloud


In [2]:
!gcloud auth application-default login

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=WK8QkSGxhwhWHUkz1drZbxVXCMOLV0&prompt=consent&token_usage=remote&access_type=offline&code_challenge=TqcuYAoiFNZxHqvGiSoDXYf_EGnm41RbHaOCfHI0hdk&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0AcvDMrAUpg5SDeIcJ8wlhFnWJp17CnMqvSRnikkyEBN7qinz1L67WoiPWecKTKSlMinShA

Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).
Ca

In [4]:
##importando pacotes
import pandas as pd
import numpy as np
import time
import os
import pandas_gbq
from google.cloud import bigquery
import glob
import openpyxl
import csv
import re

In [29]:
df = pd.read_excel('PEP_desligamentos.xlsx')
df

,Ano,Agrupamento do desligamento,Tipo de desligamento,Estabilidade,Quantidade de Desligamentos
0,2024,Decisão judicial,Decisao Judicial,Estável,2.0
1,2024,Penalidade disciplinar,"Demissao - Arts. 132 E 133, Lei 8112/90",Estável,92.0
2,2024,Penalidade disciplinar,"Demissao - Arts. 132 E 133, Lei 8112/90",Não Estável,2.0
3,2024,Penalidade disciplinar,"Demissao Art 137 Paragrafo Unico, Lei 8112/90",Estável,13.0
4,2024,Penalidade disciplinar,"Demissao Art 137,caput Lei 8112/90",Estável,14.0
...,...,...,...,...,...
460,2003,Demais desligamentos,Suspensao Temporaria Administrativa,Estável,6.0
461,NaN,NaN,NaN,NaN,NaN
462,Selection Status:,NaN,NaN,NaN,NaN
463,Sem GDF (Despesas): Sim,NaN,NaN,NaN,NaN


In [30]:
##Remover colunas com dados despadronizados devido ao sheets
df = df.drop([461, 462, 463, 464])
df

,Ano,Agrupamento do desligamento,Tipo de desligamento,Estabilidade,Quantidade de Desligamentos
0,2024,Decisão judicial,Decisao Judicial,Estável,2.0
1,2024,Penalidade disciplinar,"Demissao - Arts. 132 E 133, Lei 8112/90",Estável,92.0
2,2024,Penalidade disciplinar,"Demissao - Arts. 132 E 133, Lei 8112/90",Não Estável,2.0
3,2024,Penalidade disciplinar,"Demissao Art 137 Paragrafo Unico, Lei 8112/90",Estável,13.0
4,2024,Penalidade disciplinar,"Demissao Art 137,caput Lei 8112/90",Estável,14.0
...,...,...,...,...,...
456,2003,Posse outro cargo inacumulável,"Posse Outro Cargo Inacumulavel-Lei 8112,Art33,...",Estável,445.0
457,2003,Posse outro cargo inacumulável,"Posse Outro Cargo Inacumulavel-Lei 8112,Art33,...",Não Estável,299.0
458,2003,Programa de desligamento volun,Programa De Desligamento Voluntario - Pdv,Estável,3.0
459,2003,Programa de desligamento volun,Programa De Desligamento Voluntario - Pdv,Não Estável,3.0


In [31]:
df['Ano'] = df['Ano'].astype(int)
df['Quantidade de Desligamentos'] = df['Quantidade de Desligamentos'].astype(int)
df = df[(df['Ano']>2002) & (df['Ano'] <= 2024)]
df

,Ano,Agrupamento do desligamento,Tipo de desligamento,Estabilidade,Quantidade de Desligamentos
0,2024,Decisão judicial,Decisao Judicial,Estável,2
1,2024,Penalidade disciplinar,"Demissao - Arts. 132 E 133, Lei 8112/90",Estável,92
2,2024,Penalidade disciplinar,"Demissao - Arts. 132 E 133, Lei 8112/90",Não Estável,2
3,2024,Penalidade disciplinar,"Demissao Art 137 Paragrafo Unico, Lei 8112/90",Estável,13
4,2024,Penalidade disciplinar,"Demissao Art 137,caput Lei 8112/90",Estável,14
...,...,...,...,...,...
456,2003,Posse outro cargo inacumulável,"Posse Outro Cargo Inacumulavel-Lei 8112,Art33,...",Estável,445
457,2003,Posse outro cargo inacumulável,"Posse Outro Cargo Inacumulavel-Lei 8112,Art33,...",Não Estável,299
458,2003,Programa de desligamento volun,Programa De Desligamento Voluntario - Pdv,Estável,3
459,2003,Programa de desligamento volun,Programa De Desligamento Voluntario - Pdv,Não Estável,3


In [32]:
df[(df['Tipo de desligamento']=='Exon.Cargo Efet. Pedido/Art.34,lei 8112/90')]
df

,Ano,Agrupamento do desligamento,Tipo de desligamento,Estabilidade,Quantidade de Desligamentos
0,2024,Decisão judicial,Decisao Judicial,Estável,2
1,2024,Penalidade disciplinar,"Demissao - Arts. 132 E 133, Lei 8112/90",Estável,92
2,2024,Penalidade disciplinar,"Demissao - Arts. 132 E 133, Lei 8112/90",Não Estável,2
3,2024,Penalidade disciplinar,"Demissao Art 137 Paragrafo Unico, Lei 8112/90",Estável,13
4,2024,Penalidade disciplinar,"Demissao Art 137,caput Lei 8112/90",Estável,14
...,...,...,...,...,...
456,2003,Posse outro cargo inacumulável,"Posse Outro Cargo Inacumulavel-Lei 8112,Art33,...",Estável,445
457,2003,Posse outro cargo inacumulável,"Posse Outro Cargo Inacumulavel-Lei 8112,Art33,...",Não Estável,299
458,2003,Programa de desligamento volun,Programa De Desligamento Voluntario - Pdv,Estável,3
459,2003,Programa de desligamento volun,Programa De Desligamento Voluntario - Pdv,Não Estável,3


In [33]:
df = df.rename(columns={'Ano':'ano','Agrupamento do desligamento':'tipo_desligamento','Tipo de desligamento':'tipo_detalhado_desligamento','Estabilidade':'status_estabilidade','Quantidade de Desligamentos':'quantidade_desligamentos'})
df.head()

,ano,tipo_desligamento,tipo_detalhado_desligamento,status_estabilidade,quantidade_desligamentos
0,2024,Decisão judicial,Decisao Judicial,Estável,2
1,2024,Penalidade disciplinar,"Demissao - Arts. 132 E 133, Lei 8112/90",Estável,92
2,2024,Penalidade disciplinar,"Demissao - Arts. 132 E 133, Lei 8112/90",Não Estável,2
3,2024,Penalidade disciplinar,"Demissao Art 137 Paragrafo Unico, Lei 8112/90",Estável,13
4,2024,Penalidade disciplinar,"Demissao Art 137,caput Lei 8112/90",Estável,14


In [34]:
df['status_estabilidade'].unique()

array(['Estável', 'Não Estável'], dtype=object)

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 461 entries, 0 to 460
Data columns (total 5 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   ano                          461 non-null    int64 
 1   tipo_desligamento            461 non-null    object
 2   tipo_detalhado_desligamento  461 non-null    object
 3   status_estabilidade          461 non-null    object
 4   quantidade_desligamentos     461 non-null    int64 
dtypes: int64(2), object(3)
memory usage: 18.1+ KB


In [36]:
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano de referencia da informacao'),
 bigquery.SchemaField('tipo_desligamento','STRING',description='Qual o agrupamento do tipo de desligamento'),
 bigquery.SchemaField('tipo_detalhado_desligamento','STRING',description='Detalhes sobre o total de desligamentos'),
 bigquery.SchemaField('status_estabilidade','STRING',description='Se esses servidores possuem estabilidade ou não'),
 bigquery.SchemaField('quantidade_desligamentos','FLOAT',description='Quantidade de desligamentos')
 ]

In [37]:
## Subindo para datalake
client = bigquery.Client(project='repositoriodedadosgpsp')
dataset_ref = client.dataset('atracao_pre_selecao')

/usr/local/lib/python3.10/dist-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [38]:
table_ref = dataset_ref.table('PEP_GF_desligamentos_v3') # nome da tabela no padrão FONTE_algo_intuitivo_dado
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
job.result()

LoadJob<project=repositoriodedadosgpsp, location=US, id=d27b0ec1-92dc-48f1-81b7-494192ce8228>